<a href="https://colab.research.google.com/github/Itiel-z/BIA-Projects/blob/main/Project_2/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 2 - Itiel

In [75]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [76]:
# Load the datasets
stock_prices = pd.read_csv('cleaned_historical_stock_prices.csv')
stocks = pd.read_csv('cleaned_historical_stocks.csv')

In [77]:
# Confirm that the stock_prices dataset has been loaded
stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,decade
0,PFE,0.846354,0.859375,0.000628,0.846354,0.859375,2332800.0,1970
1,PFE,0.859375,0.848958,0.000620,0.846354,0.861979,1531200.0,1970
2,PFE,0.848958,0.846354,0.000618,0.841146,0.851562,2452800.0,1970
3,PFE,0.846354,0.864583,0.000631,0.843750,0.869792,2227200.0,1970
4,PFE,0.864583,0.864583,0.000631,0.851562,0.864583,979200.0,1970


In [78]:
# Confirm that the stocks dataset has been loaded
stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [79]:
# Merge the stock_prices df to include sector
merged = stock_prices.merge(stocks[['ticker', 'sector']], on='ticker', how='left')

In [80]:
merged.head()

,ticker,open,close,adj_close,low,high,volume,decade,sector
0,PFE,0.846354,0.859375,0.000628,0.846354,0.859375,2332800.0,1970,HEALTH CARE
1,PFE,0.859375,0.848958,0.000620,0.846354,0.861979,1531200.0,1970,HEALTH CARE
2,PFE,0.848958,0.846354,0.000618,0.841146,0.851562,2452800.0,1970,HEALTH CARE
3,PFE,0.846354,0.864583,0.000631,0.843750,0.869792,2227200.0,1970,HEALTH CARE
4,PFE,0.864583,0.864583,0.000631,0.851562,0.864583,979200.0,1970,HEALTH CARE


In [81]:
# Implement price capping to all the price related columns
# Price columns for capping
price_columns = ['open', 'high', 'low', 'close', 'adj_close']

# Determine the bounds for each decade
for decade, group in merged.groupby('decade'):
    for col in price_columns:
        Q1 = group[col].quantile(0.25)
        Q3 = group[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Get index of this decade's rows
        decade_index = group.index

        # Cap values using to the lower or upper bounds
        merged.loc[decade_index, col] = np.clip(
            merged.loc[decade_index, col],lower_bound,upper_bound)

In [82]:
# Logical price checks
errors = merged[
    (merged['low'] > merged['high']) |
    (merged['open'] > merged['high']) |
    (merged['close'] > merged['high']) |
    (merged['open'] < merged['low']) |
    (merged['close'] < merged['low'])
]
print(f" Price consistency errors found: {len(errors)}")
display(errors)

 Price consistency errors found: 520


,ticker,open,close,adj_close,low,high,volume,decade,sector
457,GTY,0.206856,0.206856,0.001070,0.190104,0.195312,12400.0,1970,FINANCE
460,GTY,0.206856,0.206856,0.001070,0.190104,0.195312,48100.0,1970,FINANCE
461,GTY,0.206856,0.206856,0.001070,0.190104,0.195312,23400.0,1970,FINANCE
464,GTY,0.206856,0.206856,0.001070,0.190104,0.195312,27300.0,1970,FINANCE
465,GTY,0.206856,0.206856,0.001070,0.190104,0.195312,2200.0,1970,FINANCE
...,...,...,...,...,...,...,...,...,...
256188,MRNS,67.745499,70.181801,64.965760,69.090897,70.909103,18700.0,2010,HEALTH CARE
256326,MRNS,70.881798,67.345497,64.965760,67.345497,67.509102,37950.0,2010,HEALTH CARE
256390,MRNS,67.345497,64.718201,64.718201,64.636398,66.590897,23650.0,2010,HEALTH CARE
256505,MRNS,63.881802,67.045403,64.965760,64.181801,67.045403,18150.0,2010,HEALTH CARE


In [83]:
# Fix the errors
# Fix: open > high by adjusting high to open
mask = merged['open'] > merged['high']
merged.loc[mask, 'high'] = merged.loc[mask, 'open']

# Fix: close > high by adjusting high to close
mask = merged['close'] > merged['high']
merged.loc[mask, 'high'] = merged.loc[mask, 'close']

# Fix: open < low by adjusting low to open
mask = merged['open'] < merged['low']
merged.loc[mask, 'low'] = merged.loc[mask, 'open']

# Fix: close < low by adjusting low to close
mask = merged['close'] < merged['low']
merged.loc[mask, 'low'] = merged.loc[mask, 'close']

# Fix: low > high by unify to avoid inconsistency
mask = merged['low'] > merged['high']
merged.loc[mask, 'low'] = merged.loc[mask, 'high']

In [84]:
# 7-day and 20-day rolling averages of close price
merged['ma_7'] = merged['close'].rolling(window=7).mean()
merged['ma_20'] = merged['close'].rolling(window=20).mean()

# Daily return
merged['daily_return'] = merged['close'].pct_change()

# 7-day and 20-day rolling standard deviation (volatility)
merged['volatility_7'] = merged['daily_return'].rolling(window=7).std()
merged['volatility_20'] = merged['daily_return'].rolling(window=20).std()

In [85]:
# Technical indicators (Relative Strength Indicator - RSI)
delta = merged['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss

In [86]:
# Gap Feature (Open vs. Previous Close)
merged['prev_close'] = merged['close'].shift(1)
merged['gap'] = merged['open'] - merged['prev_close']

In [87]:
# Drop the NaN that resulted from the moving average
merged.dropna(inplace=True)

In [88]:
# Select the features to standardize
features_2_standardize = ['open', 'high', 'low', 'close', 'volume', 'adj_close', 'ma_7', 'ma_20', 'volatility_7', 'volatility_20', 'gap']

# Standardize the features
scaler = StandardScaler()
merged[features_2_standardize] = scaler.fit_transform(merged[features_2_standardize])

# Encode the categorical column
merged = pd.get_dummies(merged, columns=['sector'])

In [89]:
# Verify that the features were added and encoding done
print(merged.head().to_string())

   ticker      open     close  adj_close       low      high    volume  decade      ma_7     ma_20  daily_return  volatility_7  volatility_20  prev_close       gap  sector_BASIC INDUSTRIES  sector_CAPITAL GOODS  sector_CONSUMER DURABLES  sector_CONSUMER NON-DURABLES  sector_CONSUMER SERVICES  sector_ENERGY  sector_FINANCE  sector_HEALTH CARE  sector_MISCELLANEOUS  sector_PUBLIC UTILITIES  sector_TECHNOLOGY  sector_TRANSPORTATION
20    PFE -1.004315 -1.003518  -0.944818 -1.001295 -1.006879 -0.133030    1970 -2.009363 -2.416307      0.015015     -0.164787      -0.212927    0.867188  0.005019                    False                 False                     False                         False                     False          False           False                True                 False                    False              False                  False
21    PFE -1.003554 -1.003366  -0.944818 -1.000369 -1.005979 -0.056129    1970 -2.009406 -2.416070      0.002959     -0.164812      -0

In [91]:
# Set the features and target
X = merged.drop(columns=['ticker', 'close'])
y = merged['close']

# Split the df into train, test and validate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [92]:
# Combine features and target to save
train_data = X_train.copy()
train_data['close'] = y_train

val_data = X_val.copy()
val_data['close'] = y_val

test_data = X_test.copy()
test_data['close'] = y_test

# Save to CSV
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)